# get $\Delta \rho $ for differents grids

In [1]:
import numpy as np
import sys 
import os
import shutil
import matplotlib.pyplot as plt
import scipy.interpolate as sp
import ipyvolume as ipv
sys.path.append('/home/giorgiod/Documents/APDFT/dep')
sys.path.append('/home/giorgiod/Documents/APDFT/src')

In [2]:
from apdft import Calculator

In [3]:
proj_path='/home/giorgiod/MRCC_interface/PES-BS-ALCHEMY/'
os.listdir(proj_path)

['def2-QZVPP',
 'cpfl.sh',
 '631G*',
 'def2-TZVP',
 'sto3G',
 'sto6G',
 '321G',
 'cc-pVTZ',
 'aug-cc-pVQZ']

In [4]:
basisSets=[ 'def2-TZVP', 'sto3G', 'sto6G', '321G','631G*','cc-pVTZ']
molecules=['NN','CO','BF','BeNe','LiNa','HeMg','HAl']
alchemy=[-2,-1,0,1,2]
points= np.linspace(1.30,3.20,20)
def dirName(basis_set,molecule,point,alchemy=0):
    return proj_path+basis_set+'/'+molecule+"/A{}_Radius:{}".format(alchemy,str(point)[0:3])+'/'

In [5]:
def parse_energy_cc(log_file):
    """Parse the couple cluster energy from an MRCC output file"""
    try:
        with open(log_file,'r') as logf:
            while True:
                line=logf.readline()
                if "Final results:" in line:
                    good_line=logf.readline()
                    if "Total CCSD energy" in good_line:
                        for x in good_line.split(' '):
                            try:
                                float(x)
                                return (float(x))
                            except:
                                pass    
    except:
        print('couldn\'t parse energy return 0 for: '+log_file)
        return 0

In [6]:
def energy(bs,mol,pt,al=0):
    return parse_energy_cc(dirName(bs,mol,pt,al)+'run.log')

In [7]:
def get_grid(basis_set,molecule,point,alchemy=0):
    return (Calculator.MrccCalculator.get_grid(None, None, dirName(basis_set,molecule,point,alchemy=0)))

In [8]:
def get_ccdensity(basis_set,molecule,point,alchemy=0):
    return Calculator.MrccCalculator.density_on_grid(dirName(basis_set,molecule,point,alchemy=0)+'DENSITY',get_grid(basis_set,molecule,point,alchemy=0)[0])

array([[-12.33170215,  -4.28288152,  -1.93735506],
       [-12.07967597,  -3.75823928,  -3.75823928],
       [-11.65726394,  -5.41375104,  -2.99446049],
       ...,
       [ 13.95726394,   5.41375104,   2.99446049],
       [ 14.37967597,   3.75823928,   3.75823928],
       [ 14.63170215,   4.28288152,   1.93735506]])

In [15]:
def get_grid(basis_set,molecule,point,alchemy=0):
    return (Calculator.MrccCalculator.get_grid(None, None, dirName(basis_set,molecule,point,alchemy=0)))
def get_ccdensity(basis_set,molecule,point,alchemy=0):

    return Calculator.MrccCalculator.density_on_grid(dirName(basis_set,molecule,point,alchemy=0)+'DENSITY',get_grid(basis_set,molecule,point,alchemy=0)[0])
gnn,wnn=get_grid('sto3G','NN',2.3)[0],get_grid('sto3G','NN',2.3)[1]
dnn=get_ccdensity('sto3G','NN',2.3)
gbne,wbne=get_grid('sto3G','BeNe',2.3)[0],get_grid('sto3G','BeNe',2.3)[1]
dbne=get_ccdensity('sto3G','BeNe',2.3)

def Colour(a):
    colour=[]
    mina=min(a)
    maxa=max(a)
    for i in range (len(a)):
        colour.append((((a[i]-mina)/(maxa-mina))**0.5*256,0,0))#((maxa-dw[i])/(maxa-mina))**0.5*256))
        #colour.append((256,0,0))
    return colour
#ipv.quickscatter(gbne[:,0],gbne[:,1],gbne[:,2],size=0.05,color=Colour(dbne))
#ipv.show()

In [10]:
import numpy as np
import scipy
import scipy.ndimage as snd
import matplotlib.pyplot as plt

In [11]:
def resample(rho, npoints, bX,bY,bZ, X, Y, Z):
    Xs, Ys, Zs = np.mgrid[bX[0]:bX[1]:complex(npoints), bY[0]:bY[1]:complex(npoints), bZ[0]:bZ[1]:complex(npoints)]
    positions = np.vstack([Xs.ravel(), Ys.ravel(), Zs.ravel()])
    newrho = snd.map_coordinates(rho, positions, mode='nearest', order=3)
    return newrho.reshape(npoints, npoints, npoints), npoints, npoints, npoints, X*NX/npoints, Y*NY/npoints, Z*NZ/npoints  

In [28]:
a=scipy.interpolate.griddata(gbne[:],dbne[:],gnn[:],method='nearest')
b=scipy.interpolate.griddata(gnn[:],a[:],gbne[:],method='nearest')
print(np.sum(wbne*dbne))
print(np.sum(wnn*a))
print(np.sum(wnn*abs(a-dnn)))
print(np.sum(wbne*(dbne-b)))
#ipv.quickscatter(gnn[:,0],gnn[:,1],gnn[:,2],size=0.05,color=Colour(abs(a-dnn)))
#ipv.show()

13.999998443815755
14.000487120251636
9.258912438098276
0.0009760421886811385


In [25]:
# method='linear' didn't work , memory problem???
print ('grid points 1st grid=', gbne.shape)
print ('grid points 2nd grid=', gnn.shape)
"""
%time
a=scipy.interpolate.griddata(gbne[:1000],dbne[:1000],gnn[:1000],method='linear')
%time
a=scipy.interpolate.griddata(gbne[:10000],dbne[:10000],gnn[:10000],method='linear')
%time
a=scipy.interpolate.griddata(gbne[:20000],dbne[:20000],gnn[:20000],method='linear')
%time
"""
###seems not hard,though 
"""
a=scipy.interpolate.griddata(gbne[:gnn.shape[0]],dbne[:gnn.shape[0]],gnn,method='linear')
print(np.sum(wbne*dbne))
print(np.sum(wnn*a))
print(np.sum(wnn*abs(a-dnn)))
## outputs nan
grid points 1st grid= (90806, 3)
grid points 2nd grid= (58072, 3)
13.999998443815755
nan
nan
\
"""
##better use method='nearest'

grid points 1st grid= (90806, 3)
grid points 2nd grid= (58072, 3)


"\na=scipy.interpolate.griddata(gbne[:gnn.shape[0]],dbne[:gnn.shape[0]],gnn,method='linear')\nprint(np.sum(wbne*dbne))\nprint(np.sum(wnn*a))\nprint(np.sum(wnn*abs(a-dnn)))\n## outputs nan\ngrid points 1st grid= (90806, 3)\ngrid points 2nd grid= (58072, 3)\n13.999998443815755\nnan\nnan\n"

In [34]:
def delta_Rho(bs,dist,mol1,mol2): 
    gg1=get_grid(bs,mol1,dist)
    g1,w1=gg1[0],gg1[1]
    gg2=get_grid(bs,mol2,dist)
    g2,w2=gg2[0],gg2[1]    
    d1=get_ccdensity(bs,mol1,dist)
    d2=get_ccdensity(bs,mol2,dist)
    p1_2=scipy.interpolate.griddata(g1[:],d1[:],g2[:],method='nearest')
    print(np.sum(w2*abs(p1_2-d2)))
    p2_1=scipy.interpolate.griddata(g2[:],d2[:],g1[:],method='nearest')
    print(np.sum(w1*abs(p2_1-d1)))

In [41]:
delta_Rho('sto3G',3.0,'NN','NN')

0.0
0.0


In [43]:
delta_Rho('sto3G',3.0,'CO','NN')

3.277756133394246
3.277802467724164


In [44]:
delta_Rho('sto3G',3.0,'BF','NN')

6.551141689773403
6.552254276047702


In [46]:
delta_Rho('sto3G',3.0,'HAl','NN')

13.066294452538452
12.806900862978585


In [47]:
delta_Rho('cc-pVTZ',2.0,'BeNe','BF')

3.0802084526835567
3.395346620111522
